In [1]:
!pip install langchain -q
!pip install langchain_community -q
!pip install -U langchain-openai -q
!pip install chromadb -q

In [2]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
import os
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

In [3]:
load_dotenv()

openai_api_key = os.environ['OPENAI_API_KEY']

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [5]:
# Create ChromaDB retriever (assumes ChromaDB is already set up)
persist_directory = "chromadb_retriever"
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
db = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever = db.as_retriever()

In [6]:
prompt_search_query = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    ("user","Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(
    OpenAI(temperature=0, openai_api_key=openai_api_key), 
    retriever, 
    prompt_search_query)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [7]:
prompt_get_answer = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\\n\\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
])

In [8]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt_search_query)

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt_get_answer)

In [10]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [11]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []  #[HumanMessage(content="What is LangChain?"), AIMessage(content="Yes")]

while True:
    query = input("Ask a question: ")
    if query.lower() == 'exit':
        break
        
    response = retrieval_chain.invoke(
        {
            "chat_history":chat_history,
            "input":query
        }
    )
    
    print('Answer: ', response['answer'])
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response['answer']))
    #= response['chat_history']
    #print('Chat history: ', chat_history)
    print()

Ask a question:  what is langchain


Answer:  Langchain is a decentralized platform that aims to provide translation services using blockchain technology. It allows users to connect with translators directly, eliminating the need for intermediaries and ensuring secure and transparent transactions. The platform also offers features such as smart contracts and token incentives to facilitate the translation process.



Ask a question:  How to install LangChain?


Answer:  To install LangChain, you would typically need to follow the steps provided by the platform developers. This may involve downloading the necessary software or applications, creating an account, and setting up your profile. It's important to visit the official LangChain website or contact their customer support for specific installation instructions and any technical assistance you may need.



Ask a question:  exit


In [24]:
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models import ChatOpenAI
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
# from langchain_community.vectorstores import Chroma
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings



load_dotenv()

persist_directory = "chroma_db"
embedding = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
chroma_client = chromadb.PersistentClient(
    path="chroma_db",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)
# Initialize Chroma vector store
vector_store = Chroma(
    collection_name="langchain_store",  # The name of the collection in your Chroma DB
    client=chroma_client,  # The initialized Chroma client
    embedding_function=embedding
)

In [25]:
vector_store.search('kitty bank', search_type='similarity')

[Document(page_content='{\'id\': \'52715796284e-51\', \'text\': "dressed in Bell-Bottoms and a Hip Italian Knit Shirt,\\\\r\\\\n          Marshmallow Shoes steps inside the Narcotics office, which is\\\\r\\\\n          literally The Basement of The Station. He looks around at The\\\\r\\\\n          Area Buzzing with Activity and sees\\\\r\\\\n                         \\\\r\\\\n          ANGLE - UNDERCOVER COPS\\\\r\\\\n                         \\\\r\\\\n          at their desks. Looking less like Cops and more like unkempt\\\\r\\\\n          Hippies or Rock N\\\\\' Rollers.\\\\r\\\\n                         \\\\r\\\\n          CLOSE - RON\\\\r\\\\n                         \\\\r\\\\n          just stands there looking at all the activity.\\\\r\\\\n                         \\\\r\\\\n          CLOSE -", \'source\': \'https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/imsdb.html\'}'),
 Document(page_content="{'id': '52715796284e-109', 'text': 'KWAME TURE\\\\r\

In [23]:
chroma_client.list_collections()

[Collection(name=langchain_store)]